# Анализ рынка заведений общественного питания Москвы

Инвесторы из фонда "Shut Up and Take My Money" планируют открыть заведение общественного питания в Москве и ищут помощи в определении его типа, расположения, меню и цен. Задача - провести исследование рынка Москвы, выявить интересные особенности и представить полученные результаты, которые помогут инвесторам принять решение о выборе места. В нашем расположении датасет с информацией о заведениях общественного питания в Москве на основе данных от сервисов Яндекс Карты и Яндекс Бизнес, собранных летом 2022 года. Представленная информация имеет исключительно справочный характер и может быть добавлена пользователями или найдена в общедоступных источниках. 

**Описание данных**

Файл moscow_places.csv:

* name — название заведения;
* address — адрес заведения;
* category — категория заведения, например «кафе», «пиццерия» или «кофейня»;
* hours — информация о днях и часах работы;
* lat — широта географической точки, в которой находится заведение;
* lng — долгота географической точки, в которой находится заведение;
* rating — рейтинг заведения по оценкам пользователей в Яндекс Картах (высшая оценка — 5.0);
* price — категория цен в заведении, например «средние», «ниже среднего», «выше среднего» и так далее;
* avg_bill — строка, которая хранит среднюю стоимость заказа в виде диапазона;
* middle_avg_bill — число с оценкой среднего чека, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Средний счёт»;
* middle_coffee_cup — число с оценкой одной чашки капучино, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Цена одной чашки капучино»;
* chain — число, выраженное 0 или 1, которое показывает, является ли заведение сетевым (для маленьких сетей могут встречаться ошибки): 0 — заведение не является сетевым, 1 — заведение является сетевым;
* district — административный район, в котором находится заведение, например Центральный административный округ;
* seats — количество посадочных мест.

### Загрузка данных и изучение общей информации

In [11]:
!pip install folium

In [12]:
# импорт библиотек
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from plotly import graph_objects as go
import plotly.express as px
import json
from folium import Map, Choropleth
from folium import Map, Marker
from folium.plugins import MarkerCluster
from folium.features import GeoJsonTooltip
import folium
import plotly.subplots as sp
from folium.features import CustomIcon

In [13]:
# чтение файла
path = '/datasets/'
df = pd.read_csv(path + 'moscow_places.csv')
df.head(4)

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/moscow_places.csv'

In [ ]:
df.info()

В таблице 8406 строк, 14 столбцов. В шести столбцах присутствуют пропуски значений.

### Предобработка данных

In [ ]:
# приводим столбец с названием заведения к нижнему регистру
df['name'] = df['name'].str.lower()
df['address'] = df['address'].str.lower()

# ищем дубликаты в данных
display(df[df.duplicated(subset=['name', 'address', 'lng'], keep=False)])

# удаляем дубликаты
df = df.drop_duplicates(subset=['name', 'address', 'lng'], keep=False)

# проверка
df[df.duplicated(subset=['name', 'address', 'lng'], keep=False)]

In [ ]:
# выводим процент пропущенных значений по каждому столбцу
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

In [ ]:
# голубой - пропущенные данные, черный - не пропущенные
colours = ['#00000f', '#abfff8'] 
plt.figure(figsize=(9, 6))  # Устанавливаем размер фигуры
sns.set(font_scale=1.2)  # Устанавливаем размер шрифта
sns.heatmap(df.isnull(), cmap=sns.color_palette(colours)) # строим heatmap

Как мы видим, больше всего пропусков в столбце `middle_coffee_cup`, `middle_coffee_cup`, `price` и `avg_bill `. 

Пропуски в некоторых столбцах мы могли бы восполнить информацией из интернета или использовать среднее или медианное значение для того же типа заведения для заполнения пропусков. Но это может исказить результаты нашего исследования, поэтому оставим пропуски во всех столбцах нетронутыми.

In [ ]:
# создаем столбец с названиями улиц из столбца с адресом
df['street'] = df['address'].str.split(',').str[1].str.strip()
df['street'].head()

In [ ]:
# создаем столбец  с обозначением, что заведение работает ежедневно и круглосуточно (24/7):
# логическое значение True — если заведение работает ежедневно и круглосуточно;
# логическое значение False — в противоположном случае.
df['is_24/7'] = df['hours'].str.contains('круглосуточно', case=False)
df['is_24/7'].value_counts()

### Исследовательский анализ данных

#### Смотрим какие категории заведений представлены в данных? Исследуйте количество объектов общественного питания по категориям. Смотрим как распределяются заведения по категориям.

In [ ]:
categories = df['category'].value_counts().to_frame().reset_index()
categories = categories.rename(columns={'index': 'category', 'category': 'quantity'})
categories

In [ ]:
# Настройка цветовой палитры
color_palette = ['#3366CC', '#DC3912', '#FF9900', '#109618', '#990099', '#0099C6', '#DD4477', 'FFEA20']

# Создание круговой диаграммы
fig = go.Figure(data=[go.Pie(labels=categories['category'], values=categories['quantity'], textinfo='label+percent+value', marker=dict(colors=color_palette))])

# Настройка внешнего вида диаграммы и добавление текстовой информации
#fig.update_traces(hoverinfo='label+percent', textfont_size=12, hole=0.4)

# Настройка заголовка диаграммы
fig.update_layout(title='Распределение категорий заведений', title_font=dict(size=20), showlegend=False)
fig.update_layout(title_x=0.453)

# Отображение диаграммы
fig.show()

Всего в данных представлено 8 типов заведений общественного питания. Бар и паб объединены в одну категорию. Замечам, что больше всего у нас в данных встречаются кафе, рестораны и кофейни. Меньше всего конкуренции на рынке у владельцев столовых и булочных. Относительно небольшое количество столовых и булочных в Москве скорее всего связано с высоким уровенем доходов, что создает спрос на разнообразные рестораны и кафе.      

#### Смотрим количество посадочных мест в местах по категориям.  Визуализируем результаты. 

In [ ]:
median_seats = df.groupby('category')['seats'].median()
sorted_categories = median_seats.sort_values(ascending=False).index

# Настройка размера графика
fig, ax = plt.subplots(figsize=(13, 6))

color_palette = ['#3366CC', '#DC3912', '#FF9900', '#109618', '#990099', '#0099C6', '#DD4477', '#FF00FF']

# Построение диаграммы
sns.boxplot(x='category', y='seats', data=df, palette=color_palette, order=sorted_categories)

# Настройка осей и заголовка графика
plt.xlabel('Категория')
plt.ylabel('Количество мест')
plt.title('Количество посадочных мест по категориям заведений, отсортированных по медианному значению')

# Размещение графика по центру
plt.tight_layout()

# Отображение графика
plt.show()

In [ ]:
# Настройка размера графика
fig, ax = plt.subplots(figsize=(13, 6))

# Построение диаграммы без выбросов 
sns.boxplot(x='category', y='seats', data=df, order=sorted_categories, palette=color_palette, showfliers=False)

# Настройка осей и заголовка графика
plt.xlabel('Категория')
plt.ylabel('Количество мест')
plt.title('Количество посадочных мест по категориям заведений, отсортированных по медианному значению')

# Размещение графика по центру
plt.tight_layout()

# Ограничение оси y
plt.ylim(25, 100)

# Отображение графика
plt.show()

Из графика мы видим, что больше всего посадочных мест у ресторанов, что и логично. Меньше всего - у пиццерий и булочных, что тоже предсказуемо.

Стоит сказать, что большее количество посадочных мест позволяет одновременно обслуживать большее количество клиентов. Это особенно важно в заведениях с высокой проходимостью или в туристических районах, где может быть много посетителей одновременно. Также количество мест вслияет напрямую на рентабельность и доходы. Но количество посадочных мест несет за собой высокие затраты на аренду и обслуживание и увеличенные сложности в управлении.

#### Изображаем соотношение сетевых и несетевых заведений в датасете. Каких заведений больше?

In [ ]:
character = df['chain'].value_counts().to_frame().reset_index()
character = character.rename(columns={'index': 'type', 'chain': 'quantity'})

# Замена значений в столбце для удобства
character['type'] = character['type'].replace({0: 'Несетевые', 1: 'Сетевые'})

# Настройка цветовой палитры
color_palette = ['#005EFF', '#FF00CD']

# Создание круговой диаграммы
fig = go.Figure(data=[go.Pie(labels=character['type'], values=character['quantity'], textinfo='label+percent+value', marker=dict(colors=color_palette))])

# Настройка заголовка диаграммы
fig.update_layout(title='Соотношение сетевых и несетевых заведений', title_font=dict(size=20), showlegend=False)
fig.update_layout(title_x=0.5)

# Отображение диаграммы
fig.show()

Диаграмма показывает, что несетевых в 1,6 раз больше чем сетевых заведений.

Стоит сказать, что сетевым заведениям в некоторой степени проще конкурировать, поскольку они оперируют  несколькими филиалами и могут обладать большими ресурсами для маркетинга, брендирования и привлечения клиентов. 

#### Сморим какие категории заведений чаще являются сетевыми? 

In [ ]:
chain_inst = df[df['chain']==1]['category'].value_counts().to_frame().reset_index() \
                .rename(columns={'index': 'category', 'category': 'quantity_chain'})

total = df['category'].value_counts().to_frame().reset_index() \
                .rename(columns={'index': 'category', 'category': 'quantity_total'})

merged_table = chain_inst.merge(total, on='category')

# процент сетевых заведений
merged_table['per_chains'] = round(merged_table['quantity_chain']/merged_table['quantity_total']*100, 2)

display(merged_table.sort_values(by='per_chains', ascending=False))

fig = px.bar(chain_inst, x='category', y='quantity_chain', title='Количество сетевых заведений, разделенных по категориям', text='quantity_chain', color_discrete_sequence=['#005EFF'])

# Настройка осей
fig.update_xaxes(title='Категория заведения')
fig.update_yaxes(title='Количество заведений')

fig.show() 

Среди сетевых заведений больше всего кафе, ресторанов и кофейеен, столовы - меньше всего.

Посмотрим в процентном соотношении в каждой категории сколько сетевых и несетевых заведений.

In [ ]:
chain_df = df.groupby(['category', 'chain']).agg({'name': 'count'}).reset_index().rename(columns={'name': 'quantity'}).sort_values(by='quantity', ascending=False)
chain_df['chain'] = chain_df['chain'].replace({0: 'Несетевые', 1: 'Сетевые'})

# Расчет процентов от общей суммы
chain_df['total_quantity'] = chain_df.groupby('category')['quantity'].transform('sum')
chain_df['percentage'] = chain_df['quantity'] / chain_df['total_quantity'] * 100

# Округление процентов до двух знаков после запятой
chain_df['percentage'] = chain_df['percentage'].round(2)

# Создание графика
color_palette = ['#005EFF', '#FF00CD']
fig = px.bar(chain_df, x='category', y='quantity', color='chain', barmode='stack', text='percentage', color_discrete_sequence=color_palette)

# Настройка внешнего вида
fig.update_layout(
    title={
        'text': 'Категории заведений, которые чаще являются сетевыми',
        'xanchor': 'left',
        'yanchor': 'top',
        'x': 0
    },
    xaxis_title='Категория заведения',
    yaxis_title='Количество заведений',
    width=800,
    height=600
)

fig.update_traces(texttemplate='%{text}%')

fig.show()

Из графика видно, что среди булочных и пиццерий больше сетевых, нежели несетевых. Напротив, у баров/пабов и столовых сетевых меньше, чем несетевых. У кофеен пропорции сетевых и несетевых заведений примерно одиннаковые.

#### Группируем данные по названиям заведений и находим топ-15 популярных сетей в Москве.  Смотрим к какой категории заведений они относятся.

In [ ]:
top_15_chains = df[df['chain']==1].groupby(['name', 'category']).agg({'category': 'count'}).rename(columns={'category': 'quantity'}) \
        .sort_values(by='quantity', ascending=False).reset_index().head(15)
top_15_chains

Все сети нам хорошо знакомы. 

In [ ]:
color_palette = ['#3366CC', '#DC3912', '#FF9900', '#109618', '#990099', '#0099C6', '#DD4477', '#FF00FF']

# Создаем столбчатую диаграмму
fig = px.bar(top_15_chains, x='name', y='quantity', labels={'name': 'Название заведений', \
     'quantity': 'Количество заведений сети'}, title='Топ-15 популярных сетей в Москве', color='category', text='quantity', color_discrete_sequence=color_palette)

# Отображаем график
fig.show()

Топ-15 популярных сетей чаще относятся к категории кофейня. Наверно это из-за того, что кофейни являются популярными местами для встреч. Многие кофейни также предлагают бесплатный Wi-Fi, что делает их привлекательными для людей, которым необходимо работать или общаться вне офиса или дома. Только одно заведение из топ-15 сетей явзяется булочной.

#### Какие административные районы Москвы присутствуют в датасете? 

In [ ]:
# Заменим в столбце district словосочетание "административный округ" на "адм. округ" для удобства отображения
df_copy = df.copy()
df_copy['district'] = df_copy['district'].str.replace('административный округ', 'АО')

# Группировка данных по районам и категориям и получение суммы количества заведений
adm_districts  = df_copy.groupby(['district', 'category']).agg({'address': 'count'}).reset_index().rename(columns={'address': 'quantity'})
display(adm_districts)

color_palette = ['#3366CC', '#DC3912', '#FF9900', '#109618', '#990099', '#0099C6', '#DD4477', '#FF00FF']

# Создание графика
fig = px.bar(adm_districts, x='district', y='quantity', color='category', barmode='stack', text='quantity', color_discrete_sequence=color_palette)

# Настройка внешнего вида графика
fig.update_layout(
    title={
        'text': 'Количество заведений по районам и категориям',
        'xanchor': 'left',
        'yanchor': 'top',
        'x': 0
    },
    xaxis_title='Районы',
    yaxis_title='Количество заведений',
    width=800,
    height=600
)

# Увеличение размера графика
fig.update_layout(
    width=1000,
    height=800
)

fig.show()

Всего в данных присутствуют 9 административных районов. По общему количеству заведений лидирует Центральный АО, а меньшее количество заведений находится в Северо-Западном АО. По всем категориям заведений («кафе», «пиццерия» или «кофейня») лидирует Центральный АО. Если смотреть распределение по категориям в целом, то оно выглядит более менее пропорциональным количеству заведений в районах: где больше общее количество заведений, там будет больше ресторанов.

#### Визуализируем распределение средних рейтингов по категориям заведений. 

Посмотрим есть ли аномальные рейтинги.

In [ ]:
# Настройка размера графика
fig, ax = plt.subplots(figsize=(12, 6))

color_palette = ['#3366CC', '#DC3912', '#FF9900', '#109618', '#990099', '#0099C6', '#DD4477', '#FF00FF']

# Построение диаграммы
sns.boxplot(x='category', y='rating', data=df, palette=color_palette)

# Настройка осей и заголовка графика
plt.xlabel('Категория')
plt.ylabel('Средние рейтинги заведений')
plt.title('Распределение средних рейтингов по категориям заведений')

# Поворот названий оси x
plt.xticks(rotation=45)

# Размещение графика по центру
plt.tight_layout()

plt.show()

Видим, что у всех категорий есть заведения с не привычно низкими оценками. Поэтому возьмем функцию median().

In [ ]:
mean_rat_category  = df.groupby('category').agg({'rating': 'median'}).reset_index().rename(columns={'rating': 'mean_rating'}).sort_values(by='mean_rating',ascending=False)

# Создание графика 
fig = px.bar(mean_rat_category, x='category', y='mean_rating', color_discrete_sequence=['#005EFF'])

# Настройка внешнего вида графика
fig.update_layout(
    title='Распределение средних рейтингов по категориям заведений',
    xaxis_title='Категория',
    yaxis_title='Средний рейтинг',
    width=800,
    height=500
)

# Добавление значений над каждым столбцом
fig.update_traces(text=round(mean_rat_category['mean_rating'], 2), textposition='auto')

# Отображение графика
fig.show()

Замечаем, что усреднённые рейтинги различаются  не сильно в разных типах общепита. Выше средний рейтинг у баров/пабов, ниже - у категории кафе.

#### Строим хороплет со средним рейтингом заведений каждого района. Границы районов Москвы хранятся в файле admin_level_geomap.geojson.

In [ ]:
rating = df_copy.pivot_table(values='rating', index=['district', 'name']).reset_index() 

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

color_palette = ['#3366CC', '#DC3912', '#FF9900', '#109618', '#990099', '#0099C6', '#DD4477', '#FF00FF']

# Построение диаграммы
sns.boxplot(x='district', y='rating', data=rating, palette=color_palette)

# Настройка осей и заголовка графика
plt.xlabel('Районы')
plt.ylabel('Средние рейтинги заведений')
plt.title('Распределение средних рейтингов по районам')

# Поворот названий оси x
plt.xticks(rotation=-45)
plt.tight_layout()

plt.show()

Замечаем, что у рассматриваемых данных наблюдаются выбросы во всех  регионах. Стоит ли искать среднее?

In [ ]:
# находим для каждого района медианный рейтинг заведений
rating_df = df.groupby('district', as_index=False)['rating'].agg('median').sort_values(by='rating',ascending=False)
rating_df

In [ ]:
# читаем файл и сохраняем в переменной
with open('/datasets/admin_level_geomap.geojson', 'r') as f:
    geo_json = json.load(f)

#print(json.dumps(geo_json, indent=2, ensure_ascii=False, sort_keys=True))

# загружаем JSON-файл с границами округов Москвы
state_geo = '/datasets/admin_level_geomap.geojson'

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=rating_df,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='BuGn',
    fill_opacity=0.8,
    legend_name='Медианный рейтинг заведений по районам',
).add_to(m)

# выводим карту
m

Самый высокий медианный рейтинг имеют заведения в Центральном АО, что скорее всего связано с высокими стандартами качества. Самый низкий - в Северо-Восточном и Юго-Восточном АО. 

<div class="alert alert-success">
<b>Комментарий ревьюера: ✅</b>

Рад, что получилось визуализировать данные на карте, ведь так быстрее разобраться какая ситуация в каком округе в зависимости от показателя.   
</div>

#### Отображаем все заведения датасета на карте с помощью кластеров.

In [ ]:
# создаём карту Москвы
m1 = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m1)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
df.apply(create_clusters, axis=1)

# выводим карту
m1

Также убедились, что в центре Москвы заведеений значительно больше, чем в других районах.

#### Найдите топ-15 улиц по количеству заведений. Постройте график распределения количества заведений и их категорий по этим улицам. Попробуйте проиллюстрировать эту информацию одним графиком.

Сначала найдем топ-15 улиц по количеству заведений.

In [ ]:
# топ-15 улиц по количеству заведений
list_top_15_street = df.groupby('street').agg({'name': 'count'}).reset_index().rename(columns={'name': 'quantity'}).sort_values(by='quantity', ascending=False).head(15)['street'].tolist()
#display(top_15_street)

top_15 = df[df['street'].isin(list_top_15_street)].groupby(['street', 'category']).agg({'name': 'count'}).reset_index().rename(columns={'name': 'quantity'})#.sort_values(by='quantity', ascending=False)
#display(top_15)

# считаем общее количество заведений по всем категориям по улице
total_quant = top_15.groupby('street')['quantity'].sum().sort_values(ascending=False)
#display(total_quant)

# создаем новый столбец в оригинальном датафрейме, по которому отсортировываем значения
top_15['total_count'] = top_15['street'].map(total_quant)

# отсортировываем top_15 по total_count
top_15 = top_15.sort_values(by='total_count')
top_15

In [ ]:
color_palette = ['#3366CC', '#DC3912', '#FF9900', '#109618', '#990099', '#0099C6', '#DD4477', '#66AA00']

# Создание графика
fig = px.bar(top_15, x='quantity', y='street', color='category', barmode='stack', text='quantity', color_discrete_sequence=color_palette)

# Настройка внешнего вида графика
fig.update_layout(
    title={
        'text': 'Распределение количества заведений по топ-15 улицам и категориям',
        'xanchor': 'left',
        'yanchor': 'top',
        'x': 0
    },
    yaxis_title='Название улицы',
    xaxis_title='Общее количество заведений и по категориям',
    width=800,
    height=600
)

# Увеличение размера графика
fig.update_layout(
    width=850,
    height=600
)

fig.show() 

По построенному графику видно, что больше всего заведений находится на Проспекте Мира. Это скорее всего связано с тем, что проспект находится в центральной части Москвы и проходит через несколько районов города, а также является одной из главных магистралей города.

Среди категорий по улицам в основном лидирут рестораны и кафе. Замечаем, что на Ленинском и Ленинградском проспектах относительно мало заведений с быстрым питанием, а на МКАДе относително много кафе, но отсутствуют булочные, что и не удивительно. Самые непопулярные заведения на всех  улицах - это булочные и столовые.

#### Найдим улицы, на которых находится только один объект общепита.

Рассмотрим эти заведения в разрезе категории заведений.

In [ ]:
# улицы, на которых находится только один объект общепита
one_object = df.groupby('street').agg({'name': 'count'}).reset_index().rename(columns={'name': 'quantity'}).sort_values(by='quantity', ascending=False)
one_object = one_object[one_object['quantity']==1]['street']
display(one_object)

# единственные заведения на улицах в разрезе категории заведений
street_one_obj = df[df['street'].isin(one_object)].groupby(['street', 'category']).agg({'name': 'count'}).reset_index()['category'].value_counts().to_frame().reset_index().rename(columns={'index': 'category', 'category': 'count'})
street_one_obj

In [ ]:
#Создание графика
fig = go.Figure(data=go.Bar(x=street_one_obj['category'], y=street_one_obj['count'], text=street_one_obj['count'], marker=dict(color='#005EFF')))

# Добавление аннотаций с процентными значениями
total_count = street_one_obj['count'].sum()
for i in range(len(street_one_obj)):
    percentage = street_one_obj['count'][i] / total_count * 100
    fig.add_annotation(
        x=street_one_obj['category'][i],
        y=street_one_obj['count'][i],
        text=f"{percentage:.1f}%",
        showarrow=False,
        font=dict(size=12),
        yref='y',
        yanchor='bottom',
        ay=10
    )

# Настройка внешнего вида графика
fig.update_layout(
    title={
        'text': 'Заведения, которые являются единственными на улице, в разрезе категорий ',
        'x': 0.5,
        'y': 0.9,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Категория',
    yaxis_title='Количество заведений',
    width=700,
    height=500
)

fig.show() 

Видим, что в основном заведения на улицах с одним объектом общепита являются кафе (34.9%), рестораном (20.3%) или кофейней (18.3%). Напротив, на данных улицах редко располагаются пиццерии (3.3%) и булочные (1.7%).

Посмотрим сколько процентов среди заведений на этих улицах сетевые.

<div class="alert alert-success">
<b>Комментарий ревьюера: ✅</b>

Здорово, что визуализируешь эту информацию. Обычно её обходят стороной 😅   
</div>

In [ ]:
street_one_chain = df[df['street'].isin(one_object)]['chain'].value_counts().to_frame().reset_index().rename(columns={'index': 'is_chain', 'chain': 'count'})
street_one_chain['is_chain'] = street_one_chain['is_chain'].replace({0: 'Несетевые', 1: 'Сетевые'})
#display(street_one_chain)

# Настройка цветовой палитры
color_palette = ['#005EFF', '#FF00CD']

# Создание круговой диаграммы
fig = go.Figure(data=[go.Pie(labels=street_one_chain['is_chain'], values=street_one_chain['count'], textinfo='label+percent+value', marker=dict(colors=color_palette))])

# Настройка заголовка диаграммы
fig.update_layout(title='Соотношение сетевых и несетевых заведений у улиц с одним объектом', title_font=dict(size=20), showlegend=False)
fig.update_layout(title_x=0.5)

fig.show()

Замечаем, что у улиц с одним объектом общепита процент сетевых заведений ниже, чем в среднем по всем данным - 29% против 38.1%. Возможно, это связано с тем, что такие улицы могут быть маленькими или имеют низкий поток посетителей. Стоит учитывать этот факт при выборе размещения заведения.

<div class="alert alert-success">
<b>Комментарий ревьюера: ✅</b>

Обрати внимание, что некоторые графики у нас повторяются.
    
Это говорит о том, что мы можем создать функцию для постоянного их построения. И с помощью этого повысить оптимизацию своего кода и презентабельность проекта в целом.
    
То есть мы можем создать "обёртки" с методами, и в функцию будем подавать нужные перменные: 
    
- датасет
- подписи графика и осей
- и прочие любые настройки
    
</div>

#### Считаем медиану  столбца middle_avg_bill для каждого района (показывает примерную стоимость заказа в рублях, которая чаще всего выражена диапазоном). Строим хороплет, используя это значение в качестве ценового индикатора района для каждого района. 

In [ ]:
# Группировка данных по районам и получение медианы столбца middle_avg_bill
median_price_area  = df.groupby('district').agg({'middle_avg_bill': 'median'}).reset_index().rename(columns={'middle_avg_bill': 'median_bill'})
median_price_area.sort_values(by='median_bill', ascending=False)

In [ ]:
# создаём карту Москвы
m2 = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=median_price_area,
    columns=['district', 'median_bill'],
    #key_on='feature.name',
    key_on='feature.properties.name',
    fill_color='PuBu',
    fill_opacity=0.8,
    legend_name='Медианные значения средних чеков по районам',
).add_to(m2)

# выводим карту
m2

Замечаем, что Западный и Центральный АО имеют наибольший медианный (средний) чек по сравнению с другими районами. Напротив, Юго-Восточный, Южный и Северо-Восточный имеют наименьшие медианные (средние) чеки. Это можно объяснить потребительскими предпочтениями, а также тем, что районы с наименьшими медианными чеками могут иметь ниже уровень доходов, чем в Западном и Центральном АО, плюс они могут иметь более низкие арендные ставки для коммерческой недвижимости.

#### Исследуем особенности заведений с плохими рейтингами, средние чеки в таких местах и распределение по категориям заведений

Установим правило: заведение считается плохим, если оно получает оценку меньше или равно 3.5 баллов. Эти оценки указывают на низкое качество обслуживания, плохое качество еды или напитков и другие негативные факторы, которые не удовлетворяют клиентов.

In [ ]:
# находим заведения с плохим рейтингом
df_bad_rating = df[df['rating']<=3.5]
print('Всего {} заведения с рейтингом ниже или равно 3.5. Это составляет {}% от общего количества заведений.'.format(df_bad_rating['name'].count(), round((df_bad_rating['name'].count()/df['name'].count()*100), 3)) )

In [ ]:
median_price_area['district'] = median_price_area['district'].str.replace('административный округ', 'АО')

# смотрим медианные чеки по заведениям с плохой оценкой
median_df_bad  = df_bad_rating.groupby('district').agg({'middle_avg_bill': 'median'}).reset_index().rename(columns={'middle_avg_bill': 'median_bill_bad'})
median_df_bad['district'] = median_df_bad['district'].str.replace('административный округ', 'АО')

In [ ]:
merged_df = median_price_area.merge(median_df_bad, on='district')

merged_df['per_loss'] = (merged_df['median_bill']-merged_df['median_bill_bad'])/merged_df['median_bill']*100
display(merged_df)
print('{}% - во столько раз средний чек по всем заведениям больше среднего чека заведений с плохой оценкой, за исключением Юго-Восточного АО (там идет увеличение).'.format(round(merged_df[merged_df['district']!='Юго-Восточный АО']['per_loss'].mean())))

In [ ]:
bar_width = 0.4

# Создание массива смещений для позиционирования столбцов
bar_positions = np.arange(len(merged_df['district']))

fig, ax = plt.subplots()

# Построение столбцов для 'median_bill'
ax.bar(bar_positions, merged_df['median_bill'], width=bar_width, label=merged_df['median_bill'], color='#005EFF')

# Построение столбцов для 'median_bill_bad' смещенных вправо
ax.bar(bar_positions + bar_width, merged_df['median_bill_bad'], width=bar_width, label=merged_df['median_bill_bad'], color='#FF00CD')

# Настройка оси X
plt.xticks(bar_positions + bar_width, merged_df['district'], rotation=-45)

# Добавление значений над каждым столбцом 'median_bill'
for i in range(len(bar_positions)):
    ax.text(bar_positions[i] + bar_width / 2, merged_df['median_bill'][i], str(merged_df['median_bill'][i]), ha='center', va='bottom', fontsize=11)

# Добавление значений над каждым столбцом 'median_bill_bad'
for i in range(len(bar_positions)):
    ax.text(bar_positions[i] + 3 * bar_width / 2, merged_df['median_bill_bad'][i], str(merged_df['median_bill_bad'][i]), ha='center', va='bottom', fontsize=11)

# Настройка внешнего вида графика
plt.xlabel('Районы')
plt.ylabel('Средний (медианный) чек')
plt.title('Медианные чеки по всем данным и по заведениям с плохой оценкой')

plt.legend(['по всем заведениям', 'по заведениям с плохой оценкой'])

# Увеличение размера графика
fig.set_size_inches(10, 7)

plt.show()

У заведений с плохим рейтингом картина немного другая по сравнению с аналогичными действиями по всем данным. Центральный административный округ по среднему чеку по прежнему лидирует. Интересно, что у заведений с плохим рейтингом в Юго-Восточном АО средний чек выше, чем по другим категориям. В остальных округах средний чек у заведений с плохим рейтингом меньше в среднем на 39% по сравнению со средним рейтингом по всем заведениям.

Эти выводы логичны, поскольку заведения с более высоким рейтингом предлагают более высокое качество услуг, и соответственно, получают лучшую оценку. Клиенты, которые ожидают лучшего качества и обслуживания, могут быть готовы заплатить больше за такой опыт.

Рассмотрим распределение по улицам и категориям в местах с плохим рейтингом.  
Сначала найдем топ-15 улиц по количеству заведений.

In [ ]:
# топ-15 улиц по количеству заведений
list_top_15_street = df_bad_rating.groupby('street').agg({'name': 'count'}).reset_index().rename(columns={'name': 'quantity'}).sort_values(by='quantity', ascending=False).head(15)['street'].tolist()
#display(top_15_street)

top_15 = df_bad_rating[df_bad_rating['street'].isin(list_top_15_street)].groupby(['street', 'category']).agg({'name': 'count'}).reset_index().rename(columns={'name': 'quantity'})#.sort_values(by='quantity', ascending=False)
#display(top_15)

# считаем общее количество заведений по всем категориям по улице
total_quant = top_15.groupby('street')['quantity'].sum().sort_values(ascending=False)
#display(total_quant)

# создаем новый столбец в оригинальном датафрейме, по которому отсортировываем значения
top_15['total_count'] = top_15['street'].map(total_quant)

# отсортировываем top_15 по total_count
top_15 = top_15.sort_values(by='total_count')

# настраиваем цвета графика
color_palette = ['#3366CC', '#DC3912', '#FF9900', '#109618', '#990099', '#0099C6', '#DD4477', '#66AA00']

# Создание графика
fig = px.bar(top_15, x='quantity', y='street', color='category', barmode='stack', text='quantity', color_discrete_sequence=color_palette)

# Настройка внешнего вида графика
fig.update_layout(
    title={
        'text': 'Количество заведений с плохим рейтингом по улицам и категориям',
        'xanchor': 'left',
        'yanchor': 'top',
        'x': 0
    },
    yaxis_title='Название улицы',
    xaxis_title='Общее количество заведений  по категориям',
    width=800,
    height=600
)

# Отображение графика
fig.show() 

По графику видно, что больше всего заведений с плохим рейтингом расположены на улицах Просперт Мира и Новоясеневский проспект. Среди категории у этих заведений преобладают кафе и быстрое питание.

Как мы ранее выяснии, больше всего заведений находится на Проспекте Мира. На нем располагается широкий спектр заведений общественного питания. Среди них могут быть заведения с различными бюджетами, концепциями и уровнем обслуживания. В результате, среди такого разнообразия заведений, некоторые могут получать низкие рейтинги из-за недостаточного качества продуктов, обслуживания или других факторов.

Новоясеневский проспект находится далеко от центра. Его расположение может быть привлекательным для определенной аудитории, например, студентов или молодых людей. Такие заведения могут привлекать клиентов, которые ищут более доступные цены или имеют ограниченные финансовые возможности. Это может создать спрос на такие заведения и в результате большее количество заведений с плохим рейтингом.??правда ли

#### Промежуточный вывод

Всего в данных представлено 8 типов заведений общественного питания. Замечам, что больше всего у нас в данных встречаются кафе, рестораны и кофейни. Меньше всего конкуренции на рынке у владельцев столовых и булочных. Относительно небольшое количество столовых и булочных в Москве скорее всего связано с высоким уровенем доходов, что создает спрос на разнообразные рестораны и кафе.

Что касательно количества посадочных мест, то больше всего их у ресторанов, что и логично. Меньше всего - у пиццерий и булочных, что тоже предсказуемо. Большее количество посадочных мест позволяет одновременно обслуживать большее количество клиентов. Это особенно важно в заведениях с высокой проходимостью. Также количество мест влияет напрямую на рентабельность и доходы. Но количество посадочных мест несет за собой высокие затраты на аренду и обслуживание и увеличенные сложности в управлении.

Несетевых в 1,6 раз больше чем сетевых заведений. Стоит сказать, что сетевым заведениям в некоторой степени проще конкурировать, поскольку они оперируют несколькими филиалами и могут обладать большими ресурсами для маркетинга, брендирования и привлечения клиентов. Мы выявили, что среди булочных и пиццерий больше сетевых, нежели несетевых. Напротив, у баров/пабов и столовых сетевых меньше, чем несетевых. У кофеен пропорции сетевых и несетевых заведений примерно одиннаковые.

Топ-15 популярных сетей чаще относятся к категории кофейня. Наверно это из-за того, что кофейни являются популярными местами для встреч. Многие кофейни также предлагают бесплатный Wi-Fi, что делает их привлекательными для людей, которым необходимо работать или общаться вне офиса или дома. Только одно заведение из топ-15 сетей является булочной.

Всего в данных присутствуют 9 административных районов. По общему количеству заведений лидирует Центральный АО, а меньшее количество заведений находится в Северо-Западном АО. По всем категориям заведений («кафе», «пиццерия» или «кофейня») лидирует Центральный АО. Если смотреть распределение по категориям в целом, то оно выглядит более менее пропорциональным количеству заведений в районах: где больше общее количество заведений, там будет больше, например, ресторанов.

Замечаем, что усреднённые рейтинги различаются не сильно в разных типах общепита. Самый высокий медианный рейтинг имеют заведения в Центральном АО, что скорее всего связано с высокими стандартами качества. Самый низкий - в Северо-Восточном и Юго-Восточном АО.

Было выявлено, что больше всего заведений находится на Проспекте Мира. Это скорее всего связано с тем, что проспект находится в центральной части Москвы и проходит через несколько районов города, а также является одной из главных магистралей города. Среди категорий по улицам в основном лидируют рестораны и кафе. Замечаем, что на Ленинском и Ленинградском проспектах относительно мало заведений с быстрым питанием, а на МКАДе относително много кафе, но отсутствуют булочные. Самые непопулярные заведения на всех улицах - это булочные и столовые.

Заметили, что у улиц с одним объектом общепита процент сетевых заведений ниже, чем в среднем по всем данным - 29% против 38.1%. Возможно, это связано с тем, что такие улицы могут быть маленькими или имеют низкий поток посетителей. Стоит учитывать этот факт при выборе размещения заведения.

Замечаем, что Западный и Центральный АО имеют наибольший медианный чек по сравнению с другими районами. Напротив, Юго-Восточный, Южный и Северо-Восточный имеют наименьшие медианные чеки. Это можно объяснить потребительскими предпочтениями, а также тем, что районы с наименьшими медианными чеками могут иметь ниже уровень доходов, чем в Западном и Центральном АО, плюс они могут иметь более низкие арендные ставки для коммерческой недвижимости (и наоборот).

Мы установили, что заведение считается плохим, если оно получает оценку меньше или равно 3.5 баллов. Всего было найдено 506 заведений с рейтингом ниже или равно 3.5. Это составило 6.021% от общего количества заведений. Исследуя особенности заведений с плохими рейтингами, было замечено, что у заведений с плохим рейтингом картина немного другая по сравнению с аналогичными действиями по всем данным. Центральный административный округ по среднему чеку также лидирует. Но у заведений с плохим рейтингом в Юго-Восточном АО средний чек выше, чем по другим категориям. В остальных округах средний чек у заведений с плохим рейтингом меньше в среднем на 39% по сравнению со средним рейтингом по всем заведениям. Также было установлено, что больше всего заведений с плохим рейтингом расположены на улицах Просперт Мира и Новоясеневский проспект. Среди категории у этих заведений преобладают кафе и быстрое питание.

###  Детализация исследования: открытие кофейни

#### Сколько всего кофеен в датасете? В каких районах их больше всего, каковы особенности их расположения?

In [ ]:
# смотрим сколько всего кофеен
df_coffee_house = df[df['category']=='кофейня']
print('Всего в данных {} кофейни. Это составляет {}% от общего количества заведений.'.format(df_coffee_house.shape[0], round((df_coffee_house.shape[0]/df.shape[0]*100), 3)))

In [ ]:
# смотрим в каких районах их больше всего
coffee_house_region  = df_coffee_house.groupby('district').agg({'name': 'count'}).reset_index().rename(columns={'name': 'count'}).sort_values(by='count', ascending=False)
coffee_house_region['district'] = coffee_house_region['district'].str.replace('административный округ', 'АО')
#display(coffee_house_region)

# Создание графика
fig = go.Figure(data=go.Bar(x=coffee_house_region['district'], y=coffee_house_region['count'], text=coffee_house_region['count'], marker=dict(color='#005EFF')))

# Добавление аннотаций с процентными значениями
total_count = coffee_house_region['count'].sum()

for i in range(len(coffee_house_region)):
    percentage = coffee_house_region['count'][i] / total_count * 100
    fig.add_annotation(
        x=coffee_house_region['district'][i],
        y=coffee_house_region['count'][i],
        text=f"{percentage:.1f}%",
        showarrow=False,
        font=dict(size=12),
        yref='y',
        yanchor='bottom',
        ay=10
    )

# Настройка внешнего вида графика
fig.update_layout(
    title={
        'text': 'Количество кофеен по районам',
        'x': 0.5,
        'y': 0.9,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    yaxis_title='Количество кофеен',
    width=700,
    height=400
)

fig.show() 

Больше всего кофеен находится в Центральном АО, что логично, поскольку там находится основные бизнес центры и университеты, а также там в основном обитают туристы. Меньше всего кофеен в Юго-Западном, Юго-Восточном и Северо-Западном административных округах.

Отобразим на карте кофейни.

In [ ]:
# создаём карту Москвы
m3 = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m3)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
df_coffee_house.apply(create_clusters, axis=1)

m3

Замечаем также, что много кофеен расположены у метро и достопримечательностей, а они размещаются как раз преимущественно в центре.

#### Есть ли круглосуточные кофейни?

In [ ]:
coffee_shops_24_7 = df_coffee_house['is_24/7'].value_counts().to_frame().reset_index()
coffee_shops_24_7['index'] = coffee_shops_24_7['index'].replace({False: 'Не круглосуточно', True: 'Круглосуточно'})
print('Всего {} круглосуточных кофеен. Это {}% от всех кофеен.'.format(coffee_shops_24_7['is_24/7'][1], round(coffee_shops_24_7['is_24/7'][1]/coffee_shops_24_7['is_24/7'].sum()*100,3)))

In [ ]:
coffee_shops_24_7

In [ ]:
plt.bar(coffee_shops_24_7['index'], coffee_shops_24_7['is_24/7'], label=coffee_shops_24_7['index'], color='#005EFF')

# Настройка осей и подписей
plt.ylabel('Количество кофеен')
plt.title('Распределение кофеен по режиму работы')

# Вычисление общей суммы
total_sum = coffee_shops_24_7['is_24/7'].sum()

# Добавление процентов на столбцах
for i, v in enumerate(coffee_shops_24_7['is_24/7']):
    percent = v / total_sum * 100
    plt.text(i, v, f'{percent:.1f}%', ha='center', va='bottom', fontsize=13)

# Отображение столбчатой диаграммы
plt.tight_layout()
plt.show()

Круглосуточных кофеен более чем в 17 раз меньше, чем некруглосуточных. Возможно, если на рынке существует достаточный спрос на круглосуточные заведенения, то существует неудовлетворенные потребности потребителей. Круглосуточный режим работы может служить маркетинговым и стратегическим преимуществом вашего бизнеса, привлекая больше клиентов и создавая уникальное предложение на рынке.

#### Какие у кофеен рейтинги? Как они распределяются по районам?

In [ ]:
plt.figure(figsize=(9, 6))  # Устанавливаем размер фигуры
sns.stripplot(x='rating', y='name', data=df_coffee_house, color='#005EFF')
plt.gca().set_yticklabels([])

# Настройка осей и заголовка графика
plt.xlabel('Рейтинг')
plt.ylabel('Количество кофеен')
plt.title('Распределение рейтингов кофеен')

plt.show()

In [ ]:
for_bar = df_coffee_house.groupby('rating').agg({'name': 'count'}).reset_index()
for_bar['rating'] = for_bar['rating'].astype(str)
fig = px.bar(for_bar, x='rating', y='name', color_discrete_sequence=['#005EFF'])

# Настройка внешнего вида графика
fig.update_layout(
    title='Распределение рейтингов кофеен',
    yaxis_title='Количество кофеен',
    xaxis_title='Рейтинг',
    width=600,
    height=400
)

fig.show()

У большинства кофеен оценка пользователей находиться около 4.2-4.4. Очень мало кофеен с оценками ниже 3.7 и не так много кофеен с оценками больше 4.8.

In [ ]:
median_rat_district = df_coffee_house.groupby('district').agg({'rating': 'median'}).reset_index().sort_values(by='rating',ascending=False)
median_rat_district['district'] = median_rat_district['district'].str.replace('административный округ', 'АО')
median_rat_district

# Создание графика 
fig = px.bar(median_rat_district, x='district', y='rating', color_discrete_sequence=['#005EFF'])

# Настройка внешнего вида графика
fig.update_layout(
    title='Распределение средних рейтингов кофеен по районам',
    yaxis_title='Средний рейтинг',
    xaxis_title='Районы',
    width=600,
    height=400
)

# Добавление значений над каждым столбцом
fig.update_traces(text=round(median_rat_district['rating'], 3), textposition='auto')

fig.show()

Как мы видим из диаграммы, у всех районов средний (медианный) рейтинг кофеен почти не отличается. Только у Западного АО рейтинг немного ниже остальных.

#### На какую стоимость чашки капучино стоит ориентироваться при открытии и почему?

При условии, что мы не обращаем большое внимаение на сильную конкуренцию, нужно учитывать следующее.
Чтобы понять на какую стоимость чашки капучино стоит ориентироваться, нужно проанализировать общую ценовую политику районов Москвы. Чрезмерно низкие цены могут вызвать сомнения в качестве товаров, а слишком высокие цены могут оттолкнуть потенциальных клиентов. Стоит также проанализировать расходы на собственные затраты и вложить их в цену товаров.

In [ ]:
# Группировка данных по районам и получение медианы столбца middle_coffee_cup
median_price_area_ch = df_coffee_house.groupby('district').agg({'middle_coffee_cup': 'median'}).reset_index().rename(columns={'middle_coffee_cup': 'median_bill'})
median_price_area_ch.sort_values(by='median_bill', ascending=False)

In [ ]:
# создаём карту Москвы
m4 = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=median_price_area_ch,
    columns=['district', 'median_bill'],
    #key_on='feature.name',
    key_on='feature.properties.name',
    fill_color='PuBu',
    fill_opacity=0.8,
    legend_name='Медианные значения средних цен чашки капучино по районам',
).add_to(m4)

# выводим карту
m4

Самая низкая средняя цена на чашку капучино в Восточный АО (135 руб.), самая высокая - Юго-Западном, Центральном и Западном АО (189-198 руб.). Но скорее всего и аренда помещений в Юго-Западном и Центральном АО тоже будет самая высокая.

Рассмотрим цену средней чашки кофе в сетевых и несетевых кофейнях.

In [ ]:
# сетевые кофейни
top_15_chain = df_coffee_house[df_coffee_house['chain']==1].groupby('name').agg({'middle_coffee_cup': 'median'}).sort_values(by='middle_coffee_cup', ascending=False).reset_index().head(15)
top_15_chain

# Создание графика 
fig = px.bar(top_15_chain, x='name', y='middle_coffee_cup', color_discrete_sequence=['#005EFF'])

# Настройка внешнего вида графика
fig.update_layout(
    title='Цена средней чашки кофе в сетевых кофейнях',
    xaxis_title='Название кофейни',
    yaxis_title='Цена средней чашки кофе'
)

# Добавление значений над каждым столбцом
fig.update_traces(text=top_15_chain['middle_coffee_cup'])

# Отображение графика
fig.show()

In [ ]:
# несетевые кофейни
top_15_not_chain = df_coffee_house[df_coffee_house['chain']==0].groupby('name').agg({'middle_coffee_cup': 'median'}).sort_values(by='middle_coffee_cup', ascending=False).reset_index().head(15)
top_15_not_chain

# Создание графика 
fig = px.bar(top_15_not_chain, x='name', y='middle_coffee_cup', color_discrete_sequence=['#005EFF'])

# Настройка внешнего вида графика
fig.update_layout(
    title='Цена средней чашки кофе в несетевых кофейнях',
    xaxis_title='Название кофейни',
    yaxis_title='Цена средней чашки кофе'
)

# Добавление значений над каждым столбцом
fig.update_traces(text=top_15_not_chain['middle_coffee_cup'])

fig.show()

Из графиков видно, что цена средней чашки кофе в не
сетевых кофейнях выше, чем в сетевых. Здесь может работать масштаб экономии: сетевые кофейни могут иметь больший объем закупок, централизованное управление поставками и лучшие условия с поставщиками. Благодаря этому они могут получать скидки на ингредиенты, сырье и оборудование. В результате сетевые кофейни могут иметь более низкие себестоимости и, соответственно, могут установить более низкие цены для привлечения клиентов. 

Покажем примерные места, где можно разместить кофейню.

In [ ]:
# метро Бауманская
metro_lat, metro_lng =  55.772411, 37.679044

# создаём карту с центром
m4 = folium.Map(location=[metro_lat, metro_lng], zoom_start=16)

# сохраняем URL-адрес изображения со значком торгового центра с icons8, это путь к файлу на сервере icons8
icon_url = 'https://img.icons8.com/?size=512&id=jKUJjH7PuItN&format=png'  
# создаём объект с собственной иконкой размером 30x30
icon = CustomIcon(icon_url, icon_size=(30, 30))
# Добавление иконки на карту для метро Бауманская
folium.Marker(
    location=[metro_lat, metro_lng],
    icon=icon
).add_to(m4)

coordinates = [
    {'name': 'МГТУ им. Н.Э. Баумана', 'lat': 55.769741, 'lng': 37.691157},
    {'name': 'Первый профессиональный университет', 'lat': 55.770183, 'lng': 37.686177},
    {'name': 'Московский Государственный Областной Университет', 'lat': 55.773175, 'lng': 37.682727}
]

icon_url_un = 'https://img.icons8.com/?size=512&id=2341&format=png'

# Создание иконок для университетов
for coordinate in coordinates:
    icon = folium.features.CustomIcon(icon_url_un, icon_size=(30, 30))
    folium.Marker(
        location=[coordinate['lat'], coordinate['lng']],
        icon=icon,
        tooltip=coordinate['name'],
        popup=folium.Popup(coordinate['name'], max_width=200)
    ).add_to(m4)
       
coordinates_ch = [
    {'name': 'Место для кофейни 1', 'lat': 55.771626, 'lng': 37.680374},
    {'name': 'Место для кофейни 2', 'lat': 55.773358, 'lng': 37.683832},
    {'name': 'Место для кофейни 3', 'lat': 55.771798, 'lng': 37.681919}
]

icon_url_ch = 'https://img.icons8.com/?size=512&id=26396&format=png'

# Создание иконок для мест для кофейни
for coordinate in coordinates_ch:
    icon = folium.features.CustomIcon(icon_url_ch, icon_size=(30, 30))
    folium.Marker(
        location=[coordinate['lat'], coordinate['lng']],
        icon=icon,
        tooltip=coordinate['name'],
        popup=folium.Popup(coordinate['name'], max_width=200)
    ).add_to(m4)
# выводим карту
m4

#### Вывод

Всего в данных 1413 кофейни. Это составляет 16.813% от общего количества заведений.

Больше всего кофеен находится в Центральном АО, что логично, поскольку там находится основные бизнес центры и университеты, а также там в основном обитают туристы. Меньше всего кофеен в Юго-Западном, Юго-Восточном и Северо-Западном административных округах, а, соответственно, там  меньше конкуренции. Рассмотрев местоположение кофеен на карте, мы заметили, что много кофеен расположены у метро и достопримечательностей.

Всего 76 круглосуточных кофеен. Это 5.436% от всех кофеен. Круглосуточных кофеен более чем в 17 раз меньше, чем некруглосуточных. Возможно, если на рынке существует достаточный спрос на круглосуточные заведенения, то существует неудовлетворенные потребности потребителей. Круглосуточный режим работы может служить маркетинговым и стратегическим преимуществом бизнеса, привлекая больше клиентов. Рекомендовано сделать  дополнительное анализирование запросов, например, "круглосуточные кофейни". Также можно сделать мониторинг социальных сетей потенциальных конкурентов, чтобы узнать об неудовлетворенности потребителей. Но нужно учитывать, что для работы круглосуточной кофейни потребуются дополнительные затраты и ресурсы потребуются. Если открывать круглосуточную кофейню, то нужно выбрать такую локацию, чтобы кофейня находилаась в районе с высокой активностью или проходом людей круглосуточно.

У большинства кофеен оценка пользователей находится около 4.2-4.4. Очень мало кофеен с оценками ниже 3.7 и не так много кофеен с оценками больше 4.8. Как мы заметили из данных, у всех районов средний (медианный) рейтинг кофеен почти не отличается. Только у Западного АО рейтинг немного ниже остальных. Рейтинг аналогичных заведений указывает на то, что уровень качества и предоставляемых ими услуг находится на среднем уровне (высокой считаю рейтинг 4.5 и выше). 

При условии, что мы не обращаем большое внимаение на сильную конкуренцию, нужно учитывать следующее. Цена чашки кофе зависит от выбор региона. Чтобы понять на какую стоимость чашки капучино стоит ориентироваться, нужно ориентироваться на ценовую политику районов. Чрезмерно низкие цены могут вызвать сомнения в качестве товаров, а слишком высокие цены могут оттолкнуть потенциальных клиентов. Стоит также проанализировать расходы на собственные затраты и вложить их в цену товаров.

Самая низкая средняя цена на чашку капучино в Восточном АО (135 руб.), самая высокая - Юго-Западном, Центральном и Западном АО (189-198 руб.). Но скорее всего и аренда помещений в Юго-Западном и Центральном АО тоже будет самая высокая.

Как мы выяснили раннее, на рынке уже существуют не мало кофеен, а это может указывать на наличие спроса на такие заведения. Кафе и кофейни часто являются популярным местом для работы или учебы вне офиса или учебных заведений. Пандемия вынудила многие компании и организации перейти на удаленную работу и учебу, поэтому появилось больше потенциальных потребителей.

Рекомендовано выбирать места под кофейню с большой проходимостью вблизи оживленных районов, бизнес-центров, университетов, туристических мест и других мест с высоким потоком людей. Например, между м. Бауманская и МГТУ им. Н.Э. Баумана, или Первым профессиональным университетом, или Московским Государственным Областным Университетом. Здесь не такаядорогая аренда как в самом центре, но при этом существует постоянный поток людей. В этом случае целевой аудиторией в основном будут студенты, туристы и люди, проживающие здесь.

### Шаг – Подготовка презентации

Презентация: <https://disk.yandex.ru/i/4LmqdVTRjJMi7w>